# Notebook de descarga de información

## Información de datos abiertos

### `Explicación breve del conjunto de datos "Copiar lo que dice el conjunto de datos"`

## Importar librerias

In [17]:
# librerias base
import math
import os
import shutil 
import sys

#librerias externas
import pandas as pd 
import numpy as np
from sodapy import Socrata


# librerias propias
# Carpeta del proyecto
dir_proyecto, _ = os.path.split(os.getcwd())
sys.path.append(dir_proyecto)
#from SCRIPTS.funciones import *


In [28]:


def leer_parquet(carpeta,extension='parquet.gzip'):
    """
    

    Parameters
    ----------
    carpeta : TYPE
        DESCRIPTION.
    extension : TYPE, optional
        DESCRIPTION. The default is 'parquet.gzip'.

    Returns
    -------
    TYPE
        DESCRIPTION.

    """
    archivos_parquet = [f for f in os.listdir(f'{carpeta}') if extension in f]
    data_frames = []
    for archivo in archivos_parquet:
        data_frames.append(pd.read_parquet(os.path.join(carpeta,archivo)))
    return pd.concat(data_frames)

def dimensionar_conjunto(id_conjunto, **kwargs):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.

    Returns
    -------
    TYPE
        DESCRIPTION.

    """
    where = kwargs.get('where', '')
    client = cliente_socrata()
    conteo = client.get(id_conjunto, select = "count(1)", where = where)
    results_df = pd.DataFrame.from_records(conteo)
    return int(results_df.iloc[0,0])


def cliente_socrata(domain="www.datos.gov.co",token='grLe4ysf5HvZXuFKeZJlGZL7b'):
    """
    

    Parameters
    ----------
    domain : TYPE, optional
        DESCRIPTION. The default is "www.datos.gov.co".
    token : TYPE, optional
        DESCRIPTION. The default is 'grLe4ysf5HvZXuFKeZJlGZL7b'.

    Returns
    -------
    client : TYPE
        DESCRIPTION.

    """
    client = Socrata(domain,token)
    return client

def consultar_conjunto_paginado(id_conjunto, limit, ordenar, offset, **kwards):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.
    limit : TYPE
        DESCRIPTION.
    ordenar : TYPE
        DESCRIPTION.
    offset : TYPE
        DESCRIPTION.
    **kwards : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    where = kwards.get('where', '')
    client = cliente_socrata()
    metadatos = consultar_metadata(id_conjunto)
    results = client.get(id_conjunto, limit=limit, order=ordenar, offset=limit*offset, where =where)
    results_df = pd.DataFrame.from_records(results)#
    results_df.reindex(columns=metadatos)
    results_df.to_parquet(f"../INPUTS/Conjunto_{id_conjunto}_{offset}.parquet.gzip")

    
def consultar_metadata(id_conjunto):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.

    Returns
    -------
    columns : TYPE
        DESCRIPTION.

    """
    client = cliente_socrata()
    metadatos = client.get_metadata(id_conjunto)
    columns = [x["fieldName"] for x in metadatos["columns"]]
    return columns


## Descargar información

In [29]:
# Carpetas del proyecto

try:
    os.mkdir(os.path.join(dir_proyecto,'INPUTS'))
except:
    None

try:
    shutil.rmtree(os.path.join(dir_proyecto,'INPUTS'))
except:
    None
os.mkdir(os.path.join(dir_proyecto,'INPUTS'))


In [30]:
# parámetros de las funciones
id_conjunto = 'jbjy-vk9h'
limit = 5000
ordenar = "id_contrato"
# filtro sobre el conjunto. Año 2023 cuyo valor sea superior a cien millones de pesos
where = "fecha_de_firma>='2023-01-01T00:00:00.000' and valor_del_contrato >= 100000000"


In [31]:
tamanio_conjunto = dimensionar_conjunto(id_conjunto, where=where)
tamanio_conjunto

48359

In [32]:
loops = math.ceil(tamanio_conjunto/limit)

for offset in range (loops):
    consultar_conjunto_paginado(id_conjunto, limit, ordenar, offset, where = where)



## Probar la lectura de los datos


In [33]:
contratacion = leer_parquet(os.path.join(dir_proyecto,'INPUTS'))
contratacion

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,recursos_de_credito,recursos_propios,ultima_actualizacion,codigo_entidad,codigo_proveedor,objeto_del_contrato,fecha_inicio_liquidacion,fecha_fin_liquidacion,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion
0,ALCALDÍA DE POPAYÁN,891580006,Cauca,Popayán,"Colombia, Cauca , Popayán",Territorial,Servicio Público,Corporación Autónoma,Centralizada,CO1.BDOS.3902823,...,12483354626,0,2023-10-23T00:00:00.000,703038141,722533247,MANTENIMIENTO Y REHABILITACION DE LA MALLA VIA...,None,None,NaN,NaN
1,ALCALDIA MUNICIPIO DE VILLAVICENCIO,892099324,Meta,Villavicencio,"Colombia, Meta , Villavicencio",Territorial,No aplica/No pertenece,Ejecutivo,Centralizada,CO1.BDOS.4719581,...,0,0,2023-08-28T00:00:00.000,703915215,701231128,PRESTACION DE SERVICIO LOGISTICO DE ALIMENTACI...,2023-12-30 17:00:00,2024-04-29 17:00:00,NaN,NaN
2,SANTIAGO DE CALI DISTRITO ESPECIAL SECRETARIA...,890399011,Valle del Cauca,Cali,"Colombia, Valle del Cauca , Cali",Territorial,Inclusión Social y Reconciliación,Ejecutivo,Centralizada,CO1.BDOS.4671854,...,0,0,2023-10-11T00:00:00.000,702441189,714613239,REALIZAR LA INTERVENTORÍA TÉCNICA ADMINISTRATI...,2023-12-23 04:59:00,2024-04-23 04:59:00,NaN,NaN
3,ALCALDIA DE FLORENCIA CAUCA,800188492,Cauca,Florencia,"Colombia, Cauca , Florencia",Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,CO1.BDOS.4682641,...,0,0,2023-08-24T00:00:00.000,702224064,722743333,MEJORAMIENTO DE LAS VÍAS TERCIARIAS EN LAS VER...,None,None,NaN,NaN
4,ALCALDÍA DEL DISTRITO TURÍSTICO Y CULTURAL DE ...,890480184,Bolívar,Cartagena,"Colombia, Bolívar , Cartagena",Territorial,No aplica/No pertenece,Ejecutivo,Centralizada,CO1.BDOS.4742096,...,0,125000259,2023-08-24T00:00:00.000,703301051,711006809,AUNAR ESFUERZOS PARA BRINDAR ATENCIÓN A NIÑOS ...,2023-12-20 05:00:00,2024-04-20 04:59:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,MINISTERIO DEL INTERIOR,830114475,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,interior,Ejecutivo,Centralizada,CO1.BDOS.3935607,...,0,0,2023-10-06T00:00:00.000,700583222,713850006,Prestar servicios profesionales de asesoría se...,None,None,None,None
4996,DEFENSORÍA DEL PUEBLO,800186061,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,Servicio Público,Corporación Autónoma,Centralizada,CO1.BDOS.3832521,...,0,0,2023-10-06T00:00:00.000,702541384,709542682,Prestar los servicios profesionales para la el...,None,None,None,None
4997,DISTRITO ESPECIAL DE CIENCIA TECNOLOGIA E INNO...,890905211,Antioquia,Medellín,"Colombia, Antioquia, Medellín",Territorial,Servicio Público,Ejecutivo,No Definido,CO1.BDOS.3926158,...,0,0,2023-10-11T00:00:00.000,700221021,703264549,Arrendar el inmueble ubicado en CR 52 N 109A ...,2024-01-02 04:59:00,2024-07-01 04:59:00,None,None
4998,ICBF REGIONAL CAUCA,899999239,Cauca,Popayán,"Colombia, Cauca , Popayán",Nacional,Salud y Protección Social,Ejecutivo,Centralizada,CO1.BDOS.3933460,...,0,0,2023-10-23T00:00:00.000,704198639,706279502,PRESTAR LOS SERVICIOS DE EDUCACIÓN INICIAL EN ...,None,None,None,None
